In [18]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import pandas as pd

In [19]:
urothelial_cells=pd.read_pickle("urothelial_cell_toy_data.pkl")
images=np.transpose(urothelial_cells["X"].numpy()*255,(0,2,3,1)).astype(np.uint8)
labels=urothelial_cells["y"]

urothelial_cells.keys()
#print shape
print(images.shape)
print(urothelial_cells["X"].shape)
print(urothelial_cells["y"].shape)

(200, 256, 256, 3)
torch.Size([200, 3, 256, 256])
(200, 256, 256)


In [20]:
COLORMAP = {
    0: (0, 0, 0),       # class 0 -> black
    1: (127, 127, 127), # class 1 -> gray
    2: (255, 255, 255), # class 2 -> white
}

def colorize_mask(mask_array):
    h, w = mask_array.shape
    colorized = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in COLORMAP.items():
        colorized[mask_array == class_id] = color
    return colorized


In [21]:

root="img_label_jpgs"

X = urothelial_cells["X"].numpy()  # (200, 3, 256, 256)
Y = urothelial_cells["y"]          # (200, 256, 256)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

splits = {
    "train": (X_train, Y_train),
    "val": (X_test, Y_test),
    "test": (X_test, Y_test),
}

#make folders
for folder in ["A", "B"]:
    for split in splits:
        split_path = os.path.join(root, folder, split)
        os.makedirs(split_path, exist_ok=True)
        for filename in os.listdir(split_path):
            file_path = os.path.join(split_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

for split, (X_data, Y_data) in splits.items():
    for i in range(len(X_data)):
        img = Image.fromarray(np.transpose(X_data[i] * 255, (1, 2, 0)).astype(np.uint8))
        color_mask = colorize_mask(Y_data[i].astype(np.uint8))

        img.save(os.path.join(root, "B", split, f"{i}.jpg"))
        Image.fromarray(color_mask).save(os.path.join(root, "A", split, f"{i}.jpg"))
